# Pre-define Search Targets

In this notebook we create the search database for the simulated ChIP-seq experiments and pre-define the search target automatically. This was used for the in-person user study. So the participants don't have to manually search for a region with the target pattern.

In [5]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Enable importing modules from the parent directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../experiments'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../server'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import json
import sqlite3

from server.config import Config
from server.database import DB

In [13]:
base = "../../"
settings_filepath = "config-user-study-simulation-3-targets.json"
window_size = 3000
target_from = 7448500
target_to   = 7451500
num_searches_to_be_predefined = 10

assert target_to - target_from == window_size

with open(os.path.join(base, settings_filepath), "r") as f:
    settings = json.load(f)

**WARNING: Calling the next cell will wipe out the existing database!**

In [25]:
from ipywidgets.widgets import Checkbox

clear_db = Checkbox(value=False, description='Clear DB (Make sure you know what you do!)')
clear_db

Checkbox(value=False, description='Clear DB (Make sure you know what you do!)')

In [28]:
db_path = os.path.join(base, settings["db_path"])

if os.path.exists(db_path) and not clear_db.value:
    print('Database already exist. Rename or delete first!')
else:
    # Clear DB!
    os.remove(db_path)
    DB(db_path=db_path, clear=True)

    with sqlite3.connect(db_path) as db:
        for search_id in range(1, num_searches_to_be_predefined + 1):
            db.execute(
                """
                    INSERT INTO
                        search(id, target_from, target_to, config)
                    VALUES
                        (?, ?, ?, ?);
                """,
                (int(search_id), int(target_from), int(target_to), json.dumps(settings)),
            )

            db.commit()